In [ ]:
from planner.users.models import User

user = User(email='user@example.com')
db_user = User.model_validate(user)

In [ ]:
# Creation
from planner.database import inject_db_session
gen = inject_db_session()
try:
    session = await anext(gen)
    session.add(db_user)
    await session.commit()  # Correctly awaiting the commit operation
finally:
    await gen.aclose()  # Ensures the session is properly closed

In [ ]:
# Read
from sqlmodel import select


gen = inject_db_session()

try:
    session = await anext(gen)
    query = select(User).where(User.uuid == db_user.uuid)
    result = await session.execute(query)
    db_user = result.scalars().first()
finally:
    await gen.aclose()

db_user

In [ ]:
# Update
from sqlmodel import update

gen = inject_db_session()

try:
    session = await anext(gen)
    query = select(User).where(User.uuid == db_user.uuid)
    result = await session.execute(query)
    db_user: User = result.scalars().first()

    db_user.first_name = 'Joe'
    query = update(User).where(User.uuid == db_user.uuid).values(db_user.model_dump())
    await session.execute(query)
    await session.commit()
finally:
    await gen.aclose()

In [ ]:
# Delete
from sqlmodel import delete

gen = inject_db_session()

try:
    session = await anext(gen)
    query = delete(User).where(User.uuid == db_user.uuid)
    await session.execute(query)
    await session.commit()
finally:
    await gen.aclose()